## Calendar


In [2]:

from src.data import all_functions as af
import pandas as pd


In [9]:
import requests
import pandas as pd
from src.data import all_functions as af

PLAYER_COLUMN = "PLAYER"
TEAM_KEY = "team_key"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"
MEAN_ROW = "mean"
NAME_KEY = "name"
LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"
TOTAL_ROW = "total"
INDEX_COLUMN = "index"
NBA_NAME_LA_CLIPPERS = "LA Clippers"
YAHOO_NAME_LA_CLIPPERS = "Los Angeles Clippers"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe

def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """
    Function finds the Yahoo Fantasy player ids and player names rostered by a team in the league
    param team_dict (dictionary): dictionary containing information about a owners team
    return player_id_name_team_list (list): Yahoo Fantasy player ids and names
    return player_name_list (list): Yahoo Fantasy player names
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >=
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <=
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        try:
            player_games_dictionary[player] = \
                team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

def get_team_player_game_dataframe(team_dict, week="next", season=2020):
    """

    @param team_dict:
    @param week:
    @param season:
    @return:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season) # Run once only
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    return filtered_games_week, player_games_dictionary

In [10]:
filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(af
                                                                            .NUNN_OF_YALL_BETTA, week="current", season=2020)

[2021-02-22 22:40:02,632 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:40:02,636 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1842.7591905593872
[2021-02-22 22:40:02,637 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:40:03,178 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:40:03,182 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1843.3057100772858
[2021-02-22 22:40:03,182 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:40:07,472 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:40:07,474 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1847.5978558063507
[2021-02-22 22:40:07,475 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:40:08,423 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:40:08,425 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1848.548546075821
[2021-02-22 22:40:08,426 DEBUG] [yahoo_oauth.oaut

In [11]:
filtered_games_week

,gdte,gid,home,home_tid,visitors,visitors_tid
168,2021-02-22,0022000473,Indiana Pacers,1.610613e+09,San Antonio Spurs,1.610613e+09
169,2021-02-22,0022000474,Houston Rockets,1.610613e+09,Chicago Bulls,1.610613e+09
170,2021-02-22,0022000476,Dallas Mavericks,1.610613e+09,Memphis Grizzlies,1.610613e+09
171,2021-02-22,0022000475,Oklahoma City Thunder,1.610613e+09,Miami Heat,1.610613e+09
172,2021-02-22,0022000477,Phoenix Suns,1.610613e+09,Portland Trail Blazers,1.610613e+09
173,2021-02-22,0022000478,Utah Jazz,1.610613e+09,Charlotte Hornets,1.610613e+09
174,2021-02-22,0022000479,Los Angeles Lakers,1.610613e+09,Washington Wizards,1.610613e+09
175,2021-02-23,0022000480,Cleveland Cavaliers,1.610613e+09,Atlanta Hawks,1.610613e+09
176,2021-02-23,0022000481,Orlando Magic,1.610613e+09,Detroit Pistons,1.610613e+09
177,2021-02-23,0022000482,Brooklyn Nets,1.610613e+09,Sacramento Kings,1.610613e+09


In [13]:
player_games_dictionary



{'Patty Mills': 3,
 'Gary Trent Jr.': 3,
 'Malcolm Brogdon': 4,
 'Jimmy Butler': 4,
 'Jae Crowder': 4,
 'Duncan Robinson': 4,
 'Wendell Carter Jr.': 4,
 'Enes Kanter': 3,
 'Kyle Kuzma': 4,
 'Cameron Johnson': 4,
 'Marcus Smart': 4,
 'Khris Middleton': 3,
 'Nikola Jokic': 3,
 'Pascal Siakam': 4,
 'Jeremy Lamb': 4}

In [18]:
season_games_dataframe = get_game_information_in_season(2020)
fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
    (season_games_dataframe, week_start_date, week_end_date)
player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(af
                                                                            .NUNN_OF_YALL_BETTA)

[2021-02-22 22:44:53,890 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:44:53,893 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2134.016390323639
[2021-02-22 22:44:53,894 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:44:55,725 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:44:55,728 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2135.8518550395966
[2021-02-22 22:44:55,729 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [24]:
filtered_filtered_games_week = filtered_games_week[filtered_games_week["gdte"] == "2021-02-22"]

In [27]:
team_game_counts = filtered_filtered_games_week[HOME_KEY].append\
    (filtered_filtered_games_week[VISITORS_KEY]).value_counts()
team_game_counts

San Antonio Spurs         1
Utah Jazz                 1
Oklahoma City Thunder     1
Miami Heat                1
Portland Trail Blazers    1
Washington Wizards        1
Phoenix Suns              1
Houston Rockets           1
Los Angeles Lakers        1
Indiana Pacers            1
Chicago Bulls             1
Charlotte Hornets         1
Dallas Mavericks          1
Memphis Grizzlies         1
dtype: int64

In [29]:
player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(af
                                                                            .NUNN_OF_YALL_BETTA)

[2021-02-22 22:49:32,068 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:32,071 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2412.1949574947357
[2021-02-22 22:49:32,072 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [30]:
player_games_dictionary = get_player_games(team_game_counts, player_name_list)

[2021-02-22 22:49:40,767 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:40,771 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2420.8945808410645
[2021-02-22 22:49:40,772 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:49:41,714 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:41,716 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2421.840026140213
[2021-02-22 22:49:41,717 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:49:42,768 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:42,772 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2422.8954632282257
[2021-02-22 22:49:42,773 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 22:49:43,790 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 22:49:43,793 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2423.916601419449
[2021-02-22 22:49:43,794 DEBUG] [yahoo_oauth.oauth

{'Patty Mills': 1,
 'Gary Trent Jr.': 1,
 'Malcolm Brogdon': 1,
 'Jimmy Butler': 1,
 'Jae Crowder': 1,
 'Duncan Robinson': 1,
 'Wendell Carter Jr.': 1,
 'Enes Kanter': 1,
 'Kyle Kuzma': 1,
 'Cameron Johnson': 1,
 'Marcus Smart': 0,
 'Khris Middleton': 0,
 'Nikola Jokic': 0,
 'Pascal Siakam': 0,
 'Jeremy Lamb': 1}

In [35]:
team_name_list = list()
for player in player_name_list:
    team = af.yahoo_player_team_and_jersey(player)[0]
    team_name_list.append(team)

[2021-02-22 23:00:23,445 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:00:23,448 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3063.57178068161
[2021-02-22 23:00:23,449 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:00:24,343 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:00:24,347 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3064.4704389572144
[2021-02-22 23:00:24,348 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:00:25,306 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:00:25,309 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3065.4322288036346
[2021-02-22 23:00:25,310 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:00:26,241 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:00:26,244 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3066.367441177368
[2021-02-22 23:00:26,245 DEBUG] [yahoo_oauth.oauth.

In [49]:
def get_team_name_list_from_player_name_list(player_name_list):
    """

    @param player_name_list:
    @return:
    """
    team_name_list = list()
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        team_name_list.append(team)
    return player_name_list, team_name_list

def daily_get_player_games(team_game_counts_series, player_name_list, team_name_list):
    """

    @param team_game_counts_series:
    @param player_name_list:
    @param team_name_list:
    @return:
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player, team in zip(player_name_list,team_name_list):
        try:
            player_games_dictionary[player] = \
            team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

In [50]:
season_games_dataframe = get_game_information_in_season(2020)
fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
    (season_games_dataframe, week_start_date, week_end_date)
player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(af
                                                                            .NUNN_OF_YALL_BETTA)

player_name_list, team_name_list = get_team_name_list_from_player_name_list(player_name_list)

daily_game_count_list = list()
for day in list(filtered_games_week["gdte"].unique()):
    daily_dataframe = filtered_games_week[filtered_games_week["gdte"] == day]
    daily_team_game_counts = daily_dataframe[HOME_KEY].append(daily_dataframe[VISITORS_KEY])\
        .value_counts()
    player_games_dictionary = daily_get_player_games(daily_team_game_counts, player_name_list,
                                                     team_name_list)
    daily_game_count_list.append(sum(list(player_games_dictionary.values())))

[2021-02-22 23:06:20,798 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:06:20,802 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3420.925102710724
[2021-02-22 23:06:20,803 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:06:22,726 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:06:22,729 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3422.8525907993317
[2021-02-22 23:06:22,730 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:06:23,667 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:06:23,670 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3423.7932455539703
[2021-02-22 23:06:23,670 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-02-22 23:06:24,595 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-22 23:06:24,604 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3424.7272505760193
[2021-02-22 23:06:24,605 DEBUG] [yahoo_oauth.oaut

In [51]:
daily_game_count_list

[11, 6, 11, 2, 12, 4, 9]

SyntaxError: invalid syntax (<ipython-input-57-bed5aaf71820>, line 1)